In [72]:
import pyforest
import redshift_connector
import numpy as np
import pandas as pd 
import math
from statistics import mean

In [73]:
# connection to mortgage data and login 
conn = redshift_connector.connect(
    host='agency2.cgfhfuapm9mq.us-east-1.redshift.amazonaws.com',
    port = 5439,
    database='loanlevel',
    user='brochford',
    password='d$5Gjk2W'
  )

In [74]:
# query to choose the states and years we want
query = "select state , case when rc_race in ('2') then 'ASIANAMERICAN' when rc_race in ('3') then 'BLACK' when rc_race in ('10') then 'HISPANIC' when rc_race in ('5') then 'WHITE' else 'OTHER' end as rc_race_range , case when rc_origloanamt is null or rc_origloanamt<0 then -999 when rc_origloanamt<=85 then 85 when rc_origloanamt<=110 then 110 when rc_origloanamt<=125 then 125 when rc_origloanamt<=150 then 150 when rc_origloanamt<=350 then ceil((rc_origloanamt-150)/50.0)*50+150 when rc_origloanamt<=417 then 417 when rc_origloanamt<=450 then 450 when rc_origloanamt<=600 then ceil((rc_origloanamt-450)/50.0)*50+450 else 29999 end as origamt_bucket , lya , loan_purpose , case when rc_income is null or rc_income<0 then -999 when rc_income<=50 then 50 when rc_income<=300 then ceil((rc_income*1.0-50)/50.0)*50+50 when rc_income<=500 then ceil((rc_income*1.0-300)/100.0)*100+300 else 29999 end as income_bucket , case when denial_reason_1 in ('4','3','2','5','8') then 'BACKGROUND' when denial_reason_1 in ('1') then 'DEBTTOINCOME' else 'OTHER' end as denial_reason_1_range , county_code , as_of_year , sum(origloanamt) as origloanamt, sum(cast(currrpb>0 as smallint)) as itemcnt,sum(noterate*currrpb*cast(noterate>=0 as smallint))/nullif(sum(currrpb*cast(noterate>=0 as smallint)),0) as wa_noterate,sum(origterm*currrpb*cast(origterm>=0 as smallint))/nullif(sum(currrpb*cast(origterm>=0 as smallint)),0) as wa_origterm,sum(cast(action_type='1' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_1 ,sum(cast(action_type='5' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_5 ,sum(cast(action_type='3' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_3 ,sum(cast(action_type='2' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_2 ,sum(cast(action_type='7' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_7 ,sum(cast(action_type='8' as smallint)*1.0)/nullif(sum(1.0),0)*100 as pct_action_type_8 from recursiondownload.hmda_lar_view_2019 a left join (select state||county||tract as census_tract_fhfa, year as year_fhfa, lya, mintrct, dda, rural from recursiondownload.fhfa_area) as fhfaarea on a.geoid=fhfaarea.census_tract_fhfa and a.as_of_year = fhfaarea.year_fhfa where as_of_year>=2002 and as_of_year<=2014 and currrpb>0 and loan_purpose in ('1') and prop_type_hmda in ('1') and not(action_type in ('6','4')) group by as_of_year , state , rc_race_range , origamt_bucket , lya , loan_purpose , income_bucket , denial_reason_1_range , county_code order by as_of_year desc , state , rc_race_range , origamt_bucket , lya , loan_purpose , income_bucket , denial_reason_1_range , county_code"
df = pd.read_sql_query(query, conn)
df_new_14 = df.rename(columns={df.columns[0]: 'state'})

df_flip_14 = df_new_14[(df_new_14['state'] == 'FL') | (df_new_14['state'] == 'TX') | (df_new_14['state'] == 'CA') | (df_new_14['state'] == 'NY') | (df_new_14['state'] == 'IL')]

df_flip_14.to_csv('allBigStates20022014.csv')

In [75]:
#sample of the dataset
df_flip_14

,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',b'county_code',b'as_of_year',b'origloanamt',b'itemcnt',b'wa_noterate',b'wa_origterm',b'pct_action_type_1',b'pct_action_type_5',b'pct_action_type_3',b'pct_action_type_2',b'pct_action_type_7',b'pct_action_type_8'
23110,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,DEBTTOINCOME,06019,2014,83000.0,1,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23111,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,06001,2014,173000.0,3,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23112,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,06013,2014,132000.0,2,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23113,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,06067,2014,5000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0
23114,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,06071,2014,243000.0,4,None,None,25.00,0.0,75.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288181,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48423,2002,2307000.0,2,None,None,100.00,0.0,0.00,0.00,0.0,0.0
5288182,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48439,2002,30042000.0,34,None,None,85.29,0.0,2.94,11.76,0.0,0.0
5288183,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48453,2002,81445000.0,75,None,None,78.66,0.0,6.66,14.66,0.0,0.0
5288184,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48479,2002,635000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0


In [107]:
#opening up the crime data 
df_crime = pd.read_csv('crimeCounty20022014.csv')

In [122]:
df_flip_14.columns

Index([                 'state',         b'rc_race_range',
              b'origamt_bucket',                   b'lya',
                b'loan_purpose',         b'income_bucket',
       b'denial_reason_1_range',           b'county_code',
                  b'as_of_year',           b'origloanamt',
                     b'itemcnt',           b'wa_noterate',
                 b'wa_origterm',     b'pct_action_type_1',
           b'pct_action_type_5',     b'pct_action_type_3',
           b'pct_action_type_2',     b'pct_action_type_7',
           b'pct_action_type_8'],
      dtype='object')

In [123]:
df_new_14 = df_flip_14.rename(columns={df.columns[7]: 'countyCode'})
df_new_14.columns

Index([                 'state',         b'rc_race_range',
              b'origamt_bucket',                   b'lya',
                b'loan_purpose',         b'income_bucket',
       b'denial_reason_1_range',             'countyCode',
                  b'as_of_year',           b'origloanamt',
                     b'itemcnt',           b'wa_noterate',
                 b'wa_origterm',     b'pct_action_type_1',
           b'pct_action_type_5',     b'pct_action_type_3',
           b'pct_action_type_2',     b'pct_action_type_7',
           b'pct_action_type_8'],
      dtype='object')

In [124]:
df_new2 = df_new_14.rename(columns={df_new_14.columns[8]: 'year'})
df_new2.columns

Index([                 'state',         b'rc_race_range',
              b'origamt_bucket',                   b'lya',
                b'loan_purpose',         b'income_bucket',
       b'denial_reason_1_range',             'countyCode',
                         'year',           b'origloanamt',
                     b'itemcnt',           b'wa_noterate',
                 b'wa_origterm',     b'pct_action_type_1',
           b'pct_action_type_5',     b'pct_action_type_3',
           b'pct_action_type_2',     b'pct_action_type_7',
           b'pct_action_type_8'],
      dtype='object')

In [135]:
df_new2

,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,b'origloanamt',b'itemcnt',b'wa_noterate',b'wa_origterm',b'pct_action_type_1',b'pct_action_type_5',b'pct_action_type_3',b'pct_action_type_2',b'pct_action_type_7',b'pct_action_type_8'
23110,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,DEBTTOINCOME,6019,2014,83000.0,1,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23111,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6001,2014,173000.0,3,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23112,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6013,2014,132000.0,2,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23113,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6067,2014,5000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0
23114,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6071,2014,243000.0,4,None,None,25.00,0.0,75.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288181,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48423,2002,2307000.0,2,None,None,100.00,0.0,0.00,0.00,0.0,0.0
5288182,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48439,2002,30042000.0,34,None,None,85.29,0.0,2.94,11.76,0.0,0.0
5288183,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48453,2002,81445000.0,75,None,None,78.66,0.0,6.66,14.66,0.0,0.0
5288184,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48479,2002,635000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0


In [112]:
df_crime

,STCOFIPS,FIPS_ST,FIPS_CTY,YEAR,STUDYNO,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,...,VIOL,PROPERTY,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON
0,1001,1,1,2002,ICPSR Study No,"1st ed, August 2004",Crimes-County,1,44959,24424,...,NaN,NaN,0,18,35,48,172,1023,48,13
1,1001,1,1,2003,4360,"1st ed, August 2004",Crimes-County,1,46808,45749,...,NaN,NaN,3,29,34,95,326,1235,107,3
2,1001,1,1,2004,4466,"1st ed, August 2004",Crimes-County,1,47929,45898,...,NaN,NaN,0,11,38,103,358,1162,86,3
3,1001,1,1,2005,4717,"1st ed, August 2004",Crimes-County,1,48917,47669,...,NaN,NaN,0,15,28,67,356,1149,112,1
4,1001,1,1,2006,23780,"1st ed, August 2004",Crimes-County,1,50316,49052,...,NaN,NaN,1,16,49,85,343,1226,94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41113,56777,56,777,2010,33523,2,Crimes-County,3177,0,0,...,51.0,107.0,0,1,1,49,11,61,35,2
41114,56777,56,777,2011,34582,2,Crimes-County,3178,0,0,...,82.0,70.0,3,6,1,72,14,35,21,3
41115,56777,56,777,2012,35019,"1st ed, August 2004",Crimes-County,3178,0,0,...,40.0,79.0,0,3,0,37,13,45,21,8
41116,56777,56,777,2013,36117,"1st ed, August 2004",Crimes-County,3177,0,0,...,43.0,77.0,1,5,0,37,11,42,24,9


In [111]:
df_crime['STCOFIPS'].dtype

dtype('int32')

In [110]:
df_crime['STCOFIPS'] = pd.to_numeric(df_crime["STCOFIPS"], downcast='integer')

In [ ]:
df_crime.to_csv('crimeCounty20022014.csv')

In [79]:
# cleaning columns data type
df_new2[['countyCode', 'year']] = df_new2[['countyCode', 'year']].apply(pd.to_numeric, errors='coerce', axis=1)

In [31]:
#merging mortgage data with crime data
df_new = pd.merge(left=df_new2, right=df_crime, left_on=['countyCode', 'year'],right_on=['STCOFIPS', 'YEAR'], how='left')
df_new

,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,b'origloanamt',...,VIOL,PROPERTY,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON
0,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,DEBTTOINCOME,6019.0,2014.0,83000.0,...,4547.0,32535.0,59.0,183.0,1040.0,3265.0,7838.0,19863.0,4834.0,491.0
1,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6001.0,2014.0,173000.0,...,10356.0,57620.0,102.0,423.0,5039.0,4792.0,9833.0,35110.0,12677.0,326.0
2,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6013.0,2014.0,132000.0,...,3650.0,32232.0,47.0,236.0,1456.0,1911.0,6949.0,18715.0,6568.0,153.0
3,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6067.0,2014.0,5000.0,...,7452.0,40246.0,80.0,365.0,2261.0,4746.0,9004.0,24907.0,6335.0,266.0
4,CA,ASIANAMERICAN,85.0,0.0,1,-999.0,OTHER,6071.0,2014.0,243000.0,...,8281.0,54778.0,110.0,474.0,2435.0,5262.0,15504.0,28848.0,10426.0,433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408735,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48423.0,2002.0,2307000.0,...,NaN,NaN,8.0,127.0,179.0,600.0,1864.0,5682.0,602.0,46.0
1408736,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48439.0,2002.0,30042000.0,...,NaN,NaN,81.0,658.0,2805.0,4522.0,17787.0,60179.0,7755.0,466.0
1408737,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48453.0,2002.0,81445000.0,...,NaN,NaN,29.0,332.0,1224.0,2046.0,8186.0,33865.0,3444.0,211.0
1408738,TX,OTHER,29999.0,NaN,1,29999.0,OTHER,48479.0,2002.0,635000.0,...,NaN,NaN,8.0,59.0,203.0,916.0,2074.0,9288.0,920.0,86.0


In [127]:
df_new2 = df_new2[pd.to_numeric(df_new2['countyCode'], errors='coerce').notnull()]

In [132]:
df_new2['countyCode'].dtypes

dtype('int32')

In [131]:
df_new2['countyCode'] = pd.to_numeric(df_new2["countyCode"], downcast='integer')

<ipython-input-131-88464703857a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new2['countyCode'] = pd.to_numeric(df_new2["countyCode"], downcast='integer')


In [149]:
df_new2 = df_new2.rename(columns={df_new2.columns[5]:'income_bucket'})

In [151]:
df_new2['income_bucket'] = df_new2['income_bucket'].replace(-999,'N.A')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(50,'<=50K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(100,'>50K & <=100K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(150,'>100K & <=150K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(200,'>150K & <=200K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(250,'>200K & <=250K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(300,'>250K & <=300K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(400,'>300K & <=400K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(500,'>400K & <=500K')
df_new2['income_bucket'] = df_new2['income_bucket'].replace(29999,'>500K')

In [153]:
df_new2 = df_new2.rename(columns={df_new2.columns[2]:'origamt_bucket'})

In [155]:
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(85,'<=85K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(110,'>85K & <= 110K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(125,'>110K & <= 125K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(150,'>125K & <= 150K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(200,'>150K & <= 200K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(250,'>200K & <= 250K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(300,'>250K & <= 300K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(350,'>300K & <= 350K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(417,'>350K & <= 417K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(450,'>417K & <= 450K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(500,'>450K & <= 500K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(550,'>500K & <= 550K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(600,'>550 & <= 600K')
df_new2['origamt_bucket'] = df_new2['origamt_bucket'].replace(29999,'>600K')

In [156]:
df_new2

,state,b'rc_race_range',origamt_bucket,b'lya',loan_purpose,income_bucket,b'denial_reason_1_range',countyCode,year,b'origloanamt',b'itemcnt',b'wa_noterate',b'wa_origterm',b'pct_action_type_1',b'pct_action_type_5',b'pct_action_type_3',b'pct_action_type_2',b'pct_action_type_7',b'pct_action_type_8'
23110,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,DEBTTOINCOME,6019,2014,83000.0,1,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23111,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6001,2014,173000.0,3,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23112,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6013,2014,132000.0,2,None,None,0.00,0.0,100.00,0.00,0.0,0.0
23113,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6067,2014,5000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0
23114,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6071,2014,243000.0,4,None,None,25.00,0.0,75.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288181,TX,OTHER,>600K,NaN,1,>500K,OTHER,48423,2002,2307000.0,2,None,None,100.00,0.0,0.00,0.00,0.0,0.0
5288182,TX,OTHER,>600K,NaN,1,>500K,OTHER,48439,2002,30042000.0,34,None,None,85.29,0.0,2.94,11.76,0.0,0.0
5288183,TX,OTHER,>600K,NaN,1,>500K,OTHER,48453,2002,81445000.0,75,None,None,78.66,0.0,6.66,14.66,0.0,0.0
5288184,TX,OTHER,>600K,NaN,1,>500K,OTHER,48479,2002,635000.0,1,None,None,100.00,0.0,0.00,0.00,0.0,0.0


In [157]:
df_new2.to_csv('allBigStatesmortgage.csv')

In [114]:
df_pop = pd.read_excel('populationData5states.xlsx')
df_pop

,Label (Grouping),Total:,Hispanic or Latino,Not Hispanic or Latino:,Population of one race:,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,countyCode,countyName,statesFlip,Unnamed: 14
0,"Alachua County, Florida",278468,33614,244854,231856,160463,51171,562,17891,101,1668,12001,Alachua County,FL,Florida
1,"Baker County, Florida",28259,973,27286,26374,22185,3825,109,157,14,84,12003,Baker County,FL,Florida
2,"Bay County, Florida",175216,13846,161370,151674,128348,17549,795,4068,194,720,12005,Bay County,FL,Florida
3,"Bradford County, Florida",28303,1350,26953,26049,20320,5427,58,135,12,97,12007,Bradford County,FL,Florida
4,"Brevard County, Florida",606612,67907,538705,508461,430936,56498,1569,15587,482,3389,12009,Brevard County,FL,Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,"Wood County, Texas",44843,4894,39949,38160,35906,1658,227,230,1,138,48499,Wood County,TX,NaN
539,"Yoakum County, Texas",7694,4968,2726,2626,2488,55,22,46,0,15,48501,Yoakum County,TX,NaN
540,"Young County, Texas",17867,3508,14359,13786,13409,174,79,86,0,38,48503,Young County,TX,NaN
541,"Zapata County, Texas",13889,12999,890,844,796,9,7,16,0,16,48505,Zapata County,TX,NaN


In [119]:
df_pop['countyCode'] = pd.to_numeric(df_pop["countyCode"], downcast='integer')

In [120]:
df_pop['countyCode'].dtypes

dtype('int32')

In [ ]:
df_final = df_final.rename(columns={df_final.columns[-12]:'Hispanic_or_Latino',
                                   df_final.columns[-11]:'Not_Hispanic_or_Latino',
                                   df_final.columns[-10]:'Population_of_one_race',
                                   df_final.columns[-9]:'White_alone',
                                   df_final.columns[-8]:'Black_or_African_American',
                                   df_final.columns[-7]:'American_Indian_and_Alaska_Native',
                                   df_final.columns[-6]:'Asian',
                                   df_final.columns[-5]:'Native_Hawaiian_and_Other_Pacific_Islander',
                                   df_final.columns[-4]:'Other_Race'})

In [109]:
df_final = pd.merge(left=df_new, right=df_pop, left_on=['countyCode',],right_on=['countyCode'], how='left')

In [110]:
df_final = df_final.rename(columns={df_final.columns[-12]:'Hispanic_or_Latino',
                                   df_final.columns[-11]:'Not_Hispanic_or_Latino',
                                   df_final.columns[-10]:'Population_of_one_race',
                                   df_final.columns[-9]:'White_alone',
                                   df_final.columns[-8]:'Black_or_African_American',
                                   df_final.columns[-7]:'American_Indian_and_Alaska_Native',
                                   df_final.columns[-6]:'Asian',
                                   df_final.columns[-5]:'Native_Hawaiian_and_Other_Pacific_Islander',
                                   df_final.columns[-4]:'Other_Race'})

In [111]:
# create a population other column for a few races
df_final['population_other'] = df_final['American_Indian_and_Alaska_Native'] + df_final['Native_Hawaiian_and_Other_Pacific_Islander'] + df_final['Other_Race'] 

In [112]:
df_final = df_final.drop(columns=['American_Indian_and_Alaska_Native','Native_Hawaiian_and_Other_Pacific_Islander','Other_Race','Unnamed: 14'])

In [155]:
df_final_1.to_excel('Data_Viz_final_dataset.xlsx')

ValueError: This sheet is too large! Your sheet size is: 1408740, 45 Max sheet size is: 1048576, 16384

In [156]:
df_final_1.to_csv('Data_Viz_final_dataset.csv')

In [2]:
df_final = pd.read_csv('Data_Viz_final_dataset.csv')

<IPython.core.display.Javascript object>

In [67]:
df_final['year'].dtypes

dtype('int16')

In [12]:
df_final['year'] = pd.to_numeric(df_final["year"], downcast='integer')

<IPython.core.display.Javascript object>

In [22]:
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(-999,'N.A')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(50,'<=50K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(100,'>50K & <=100K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(150,'>100K & <=150K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(200,'>150K & <=200K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(250,'>200K & <=250K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(300,'>250K & <=300K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(400,'>300K & <=400K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(500,'>400K & <=500K')
df_final["b'income_bucket'"] = df_final["b'income_bucket'"].replace(29999,'>500K')

In [28]:
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(85,'<=85K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(110,'>85K & <= 110K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(125,'>110K & <= 125K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(150,'>125K & <= 150K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(200,'>150K & <= 200K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(250,'>200K & <= 250K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(300,'>250K & <= 300K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(350,'>300K & <= 350K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(417,'>350K & <= 417K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(450,'>417K & <= 450K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(500,'>450K & <= 500K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(550,'>500K & <= 550K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(600,'>550 & <= 600K')
df_final["b'origamt_bucket'"] = df_final["b'origamt_bucket'"].replace(29999,'>600K')

In [32]:
df_final

,Unnamed: 0,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,...,Total:,Hispanic_or_Latino,Not_Hispanic_or_Latino,Population_of_one_race,White_alone,Black_or_African_American,Asian,countyName,statesFlip,population_other
0,0,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,DEBTTOINCOME,6019.0,2014,...,1008654.0,540743.0,467911.0,438365.0,271889.0,44295.0,109665.0,Fresno County,CA,12516.0
1,1,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6001.0,2014,...,1682353.0,393749.0,1288604.0,1200067.0,472277.0,159499.0,540511.0,Alameda County,CA,27780.0
2,2,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6013.0,2014,...,1165927.0,314900.0,851027.0,784574.0,455421.0,97994.0,214520.0,Contra Costa County,CA,16639.0
3,3,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6067.0,2014,...,1585055.0,374434.0,1210621.0,1107837.0,650271.0,145724.0,276295.0,Sacramento County,CA,35547.0
4,4,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6071.0,2014,...,2181654.0,1170913.0,1010741.0,942341.0,566113.0,173322.0,176204.0,San Bernardino County,CA,26702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408735,1408735,TX,OTHER,>600K,NaN,1,>500K,OTHER,48423.0,2002,...,233479.0,47281.0,186198.0,178102.0,134452.0,38003.0,4129.0,Smith County,TX,1518.0
1408736,1408736,TX,OTHER,>600K,NaN,1,>500K,OTHER,48439.0,2002,...,2110640.0,620907.0,1489733.0,1410813.0,904884.0,358645.0,127783.0,Tarrant County,TX,19501.0
1408737,1408737,TX,OTHER,>600K,NaN,1,>500K,OTHER,48453.0,2002,...,1290188.0,421110.0,869078.0,818803.0,612824.0,96270.0,99660.0,Travis County,TX,10049.0
1408738,1408738,TX,OTHER,>600K,NaN,1,>500K,OTHER,48479.0,2002,...,267114.0,254354.0,12760.0,12201.0,9495.0,788.0,1300.0,Webb County,TX,618.0


In [43]:
df_final

,Unnamed: 0,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,...,Total:,Hispanic_or_Latino,Not_Hispanic_or_Latino,Population_of_one_race,White_alone,Black_or_African_American,Asian,countyName,statesFlip,population_other
0,0,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,DEBTTOINCOME,6019.0,2014,...,1008654.0,540743.0,467911.0,438365.0,271889.0,44295.0,109665.0,Fresno County,CA,12516.0
1,1,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6001.0,2014,...,1682353.0,393749.0,1288604.0,1200067.0,472277.0,159499.0,540511.0,Alameda County,CA,27780.0
2,2,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6013.0,2014,...,1165927.0,314900.0,851027.0,784574.0,455421.0,97994.0,214520.0,Contra Costa County,CA,16639.0
3,3,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6067.0,2014,...,1585055.0,374434.0,1210621.0,1107837.0,650271.0,145724.0,276295.0,Sacramento County,CA,35547.0
4,4,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6071.0,2014,...,2181654.0,1170913.0,1010741.0,942341.0,566113.0,173322.0,176204.0,San Bernardino County,CA,26702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408735,1408735,TX,OTHER,>600K,NaN,1,>500K,OTHER,48423.0,2002,...,233479.0,47281.0,186198.0,178102.0,134452.0,38003.0,4129.0,Smith County,TX,1518.0
1408736,1408736,TX,OTHER,>600K,NaN,1,>500K,OTHER,48439.0,2002,...,2110640.0,620907.0,1489733.0,1410813.0,904884.0,358645.0,127783.0,Tarrant County,TX,19501.0
1408737,1408737,TX,OTHER,>600K,NaN,1,>500K,OTHER,48453.0,2002,...,1290188.0,421110.0,869078.0,818803.0,612824.0,96270.0,99660.0,Travis County,TX,10049.0
1408738,1408738,TX,OTHER,>600K,NaN,1,>500K,OTHER,48479.0,2002,...,267114.0,254354.0,12760.0,12201.0,9495.0,788.0,1300.0,Webb County,TX,618.0


In [51]:
df = df_final[df_final.duplicated(subset=['year', 'countyCode','MURDER'])]

In [52]:
df_final['MURDER'].duplicated().sum()

1408534

In [58]:
df_final

,Unnamed: 0,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,...,Total:,Hispanic_or_Latino,Not_Hispanic_or_Latino,Population_of_one_race,White_alone,Black_or_African_American,Asian,countyName,statesFlip,population_other
0,0,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,DEBTTOINCOME,6019.0,2014,...,1008654.0,540743.0,467911.0,438365.0,271889.0,44295.0,109665.0,Fresno County,CA,12516.0
1,1,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6001.0,2014,...,1682353.0,393749.0,1288604.0,1200067.0,472277.0,159499.0,540511.0,Alameda County,CA,27780.0
2,2,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6013.0,2014,...,1165927.0,314900.0,851027.0,784574.0,455421.0,97994.0,214520.0,Contra Costa County,CA,16639.0
3,3,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6067.0,2014,...,1585055.0,374434.0,1210621.0,1107837.0,650271.0,145724.0,276295.0,Sacramento County,CA,35547.0
4,4,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6071.0,2014,...,2181654.0,1170913.0,1010741.0,942341.0,566113.0,173322.0,176204.0,San Bernardino County,CA,26702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408735,1408735,TX,OTHER,>600K,NaN,1,>500K,OTHER,48423.0,2002,...,233479.0,47281.0,186198.0,178102.0,134452.0,38003.0,4129.0,Smith County,TX,1518.0
1408736,1408736,TX,OTHER,>600K,NaN,1,>500K,OTHER,48439.0,2002,...,2110640.0,620907.0,1489733.0,1410813.0,904884.0,358645.0,127783.0,Tarrant County,TX,19501.0
1408737,1408737,TX,OTHER,>600K,NaN,1,>500K,OTHER,48453.0,2002,...,1290188.0,421110.0,869078.0,818803.0,612824.0,96270.0,99660.0,Travis County,TX,10049.0
1408738,1408738,TX,OTHER,>600K,NaN,1,>500K,OTHER,48479.0,2002,...,267114.0,254354.0,12760.0,12201.0,9495.0,788.0,1300.0,Webb County,TX,618.0


In [59]:
df_final.columns

Index(['Unnamed: 0', 'state', 'b'rc_race_range'', 'b'origamt_bucket'',
       'b'lya'', 'b'loan_purpose'', 'b'income_bucket'',
       'b'denial_reason_1_range'', 'countyCode', 'year', 'b'origloanamt'',
       'b'itemcnt'', 'b'pct_action_type_1'', 'b'pct_action_type_5'',
       'b'pct_action_type_3'', 'b'pct_action_type_2'', 'b'pct_action_type_7'',
       'b'pct_action_type_8'', 'CPOPARST', 'CPOPCRIM', 'AG_ARRST', 'AG_OFF',
       'COVIND', 'INDEX', 'MODINDX', 'VIOL', 'PROPERTY', 'MURDER', 'RAPE',
       'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY', 'MVTHEFT', 'ARSON',
       'Label (Grouping)', 'Total:', 'Hispanic_or_Latino',
       'Not_Hispanic_or_Latino', 'Population_of_one_race', 'White_alone',
       'Black_or_African_American', 'Asian', 'countyName', 'statesFlip',
       'population_other'],
      dtype='object')

In [64]:
df_final['countyCode'] = pd.to_numeric(df_final["countyCode"], downcast='integer')
#df_final["b'origloanamt"] = pd.to_numeric(df_final["b'origloanamt"], downcast='integer')
df_final['CPOPARST'] = pd.to_numeric(df_final["CPOPARST"], downcast='integer')
df_final['CPOPCRIM'] = pd.to_numeric(df_final["CPOPCRIM"], downcast='integer')
df_final['AG_ARRST'] = pd.to_numeric(df_final["AG_ARRST"], downcast='integer')
df_final['AG_OFF'] = pd.to_numeric(df_final["AG_OFF"], downcast='integer')
df_final['VIOL'] = pd.to_numeric(df_final["VIOL"], downcast='integer')
df_final['PROPERTY'] = pd.to_numeric(df_final["PROPERTY"], downcast='integer')
df_final['MURDER'] = pd.to_numeric(df_final["MURDER"], downcast='integer')
df_final['RAPE'] = pd.to_numeric(df_final["RAPE"], downcast='integer')
df_final['ROBBERY'] = pd.to_numeric(df_final["ROBBERY"], downcast='integer')
df_final['AGASSLT'] = pd.to_numeric(df_final["AGASSLT"], downcast='integer')
df_final['BURGLRY'] = pd.to_numeric(df_final["BURGLRY"], downcast='integer')
df_final['LARCENY'] = pd.to_numeric(df_final["LARCENY"], downcast='integer')
df_final['MVTHEFT'] = pd.to_numeric(df_final["MVTHEFT"], downcast='integer')
df_final['ARSON'] = pd.to_numeric(df_final["ARSON"], downcast='integer')
df_final['Total:'] = pd.to_numeric(df_final["Total:"], downcast='integer')
df_final['Hispanic_or_Latino'] = pd.to_numeric(df_final["Hispanic_or_Latino"], downcast='integer')
df_final['Not_Hispanic_or_Latino'] = pd.to_numeric(df_final["Not_Hispanic_or_Latino"], downcast='integer')
df_final['Population_of_one_race'] = pd.to_numeric(df_final["Population_of_one_race"], downcast='integer')
df_final['White_alone'] = pd.to_numeric(df_final["White_alone"], downcast='integer')
df_final['Black_or_African_American'] = pd.to_numeric(df_final["Black_or_African_American"], downcast='integer')
df_final['Asian'] = pd.to_numeric(df_final["Asian"], downcast='integer')
df_final['population_other'] = pd.to_numeric(df_final["population_other"], downcast='integer')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyError: "b'origloanamt"

In [69]:
df_final['countyCode'] = pd.to_numeric(df_final["countyCode"], downcast='integer')

<IPython.core.display.Javascript object>

In [71]:
df_final['countyCode'].astype(int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [66]:
df_final

,Unnamed: 0,state,b'rc_race_range',b'origamt_bucket',b'lya',b'loan_purpose',b'income_bucket',b'denial_reason_1_range',countyCode,year,...,Total:,Hispanic_or_Latino,Not_Hispanic_or_Latino,Population_of_one_race,White_alone,Black_or_African_American,Asian,countyName,statesFlip,population_other
0,0,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,DEBTTOINCOME,6019.0,2014,...,1008654.0,540743.0,467911.0,438365.0,271889.0,44295.0,109665.0,Fresno County,CA,12516.0
1,1,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6001.0,2014,...,1682353.0,393749.0,1288604.0,1200067.0,472277.0,159499.0,540511.0,Alameda County,CA,27780.0
2,2,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6013.0,2014,...,1165927.0,314900.0,851027.0,784574.0,455421.0,97994.0,214520.0,Contra Costa County,CA,16639.0
3,3,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6067.0,2014,...,1585055.0,374434.0,1210621.0,1107837.0,650271.0,145724.0,276295.0,Sacramento County,CA,35547.0
4,4,CA,ASIANAMERICAN,<=85K,0.0,1,N.A,OTHER,6071.0,2014,...,2181654.0,1170913.0,1010741.0,942341.0,566113.0,173322.0,176204.0,San Bernardino County,CA,26702.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408735,1408735,TX,OTHER,>600K,NaN,1,>500K,OTHER,48423.0,2002,...,233479.0,47281.0,186198.0,178102.0,134452.0,38003.0,4129.0,Smith County,TX,1518.0
1408736,1408736,TX,OTHER,>600K,NaN,1,>500K,OTHER,48439.0,2002,...,2110640.0,620907.0,1489733.0,1410813.0,904884.0,358645.0,127783.0,Tarrant County,TX,19501.0
1408737,1408737,TX,OTHER,>600K,NaN,1,>500K,OTHER,48453.0,2002,...,1290188.0,421110.0,869078.0,818803.0,612824.0,96270.0,99660.0,Travis County,TX,10049.0
1408738,1408738,TX,OTHER,>600K,NaN,1,>500K,OTHER,48479.0,2002,...,267114.0,254354.0,12760.0,12201.0,9495.0,788.0,1300.0,Webb County,TX,618.0


In [70]:
df_final['countyCode'].dtypes

dtype('float64')